In [ ]:
# Time series parameters
forecast_start: str = "2024-01-01"  # {"description": "Start date for forecast", "input_type": "date"}
forecast_periods: int = 30  # {"description": "Number of periods to forecast", "validation": {"min": 1, "max": 90}}

# Decomposition parameters
decomposition_type: str = "multiplicative"  # {"description": "Type of decomposition", "input_type": "select", "options": ["multiplicative", "additive"]}
seasonality_period: int = 7  # {"description": "Days per seasonal cycle", "validation": {"min": 1, "max": 30}}

# Model parameters
smoothing_level: float = 0.2  # {"description": "Alpha smoothing factor", "validation": {"min": 0.0, "max": 1.0}}
smoothing_seasonal: float = 0.1  # {"description": "Gamma smoothing factor", "validation": {"min": 0.0, "max": 1.0}}

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.pyplot as plt

# Set style
plt.style.use('ggplot')

# Generate sample data
np.random.seed(42)
dates = pd.date_range(start='2023-01-01', end=forecast_start, freq='D')

# Create time series with trend, seasonality, and noise
trend = np.linspace(100, 200, len(dates))
seasonal = 20 * np.sin(2 * np.pi * np.arange(len(dates)) / seasonality_period)
noise = np.random.normal(0, 10, len(dates))

if decomposition_type == 'multiplicative':
    data = trend * (1 + seasonal / 100) + noise
else:  # additive
    data = trend + seasonal + noise

# Create time series
ts = pd.Series(data, index=dates)

# Decompose the time series
decomposition = seasonal_decompose(ts, model=decomposition_type, period=seasonality_period)

# Plot decomposition
fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=(12, 10))
decomposition.observed.plot(ax=ax1)
ax1.set_title('Observed')
decomposition.trend.plot(ax=ax2)
ax2.set_title('Trend')
decomposition.seasonal.plot(ax=ax3)
ax3.set_title('Seasonal')
decomposition.resid.plot(ax=ax4)
ax4.set_title('Residual')
plt.tight_layout()
plt.show()

# Simple forecasting using the last values and seasonal components
last_trend = decomposition.trend.dropna().iloc[-1]
seasonal_components = decomposition.seasonal.iloc[-seasonality_period:]

# Extend seasonal pattern
forecast_seasonal = np.tile(seasonal_components.values, 
                           int(np.ceil(forecast_periods / seasonality_period)))[:forecast_periods]

# Create forecast
if decomposition_type == 'multiplicative':
    forecast_values = last_trend * (1 + forecast_seasonal / 100)
else:  # additive
    forecast_values = last_trend + forecast_seasonal

# Plot the forecast
plt.figure(figsize=(12, 6))
plt.plot(dates, data, label='Historical')
plt.plot(pd.date_range(start=forecast_start, 
                      periods=forecast_periods, 
                      freq='D'),
         forecast_values[-forecast_periods:],
         '--', label='Forecast')
plt.title('Sales Forecast')
plt.legend()
plt.tight_layout()
plt.show()